# Prediction Restaurant revenues usig machine learning
This projects aims at predicting the annual restaurant sales of 100,000 regional locations.

## 1. Problem Definition
> How well can we predict the revenues generated in restaurants from 100,000 regional locations?

## 2. Data
This dataset is from the Restaurant Revenue Prediction Kaggle competition. https://www.kaggle.com/competitions/restaurant-revenue-prediction/data

* train.csv - the training set. Use this dataset for training your model. 
* test.csv - the test set. To deter manual "guess" predictions, Kaggle has supplemented the test set with additional "ignored" data. These are not counted in the scoring.
* sampleSubmission.csv - a sample submission file in the correct format

## 3. Evaluation
The evaluation metric is the RMSE (root maen square error) between the original value and predicted value.
https://www.kaggle.com/competitions/restaurant-revenue-prediction/overview/evaluation

## 4. Features
This is where you'll get different information about each features in your data
**Data Dictionary**

Data fields
* Id : Restaurant id. 
* Open Date : opening date for a restaurant
* City : City that the restaurant is in. Note that there are unicode in the names. 
* City Group: Type of the city. Big cities, or Other. 
* Type: Type of the restaurant. FC: Food Court, IL: Inline, DT: Drive Thru, MB: Mobile
* P1, P2 - P37: There are three categories of these obfuscated data. Demographic data are gathered from third party providers with GIS systems. These include population in any given area, age and gender distribution, development scales. Real estate data mainly relate to the m2 of the location, front facade of the location, car park availability. Commercial data mainly include the existence of points of interest including schools, banks, other QSR operators.
* Revenue: The revenue column indicates a (transformed) revenue of the restaurant in a given year and is the target of predictive analysis. Please note that the values are transformed so they don't mean real dollar values. 

## 5. Modelling

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns

%matplotlib inline

In [3]:
df = pd.read_csv("data/train.csv",
                 parse_dates=["Open Date"])
df

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
0,0,1999-07-17,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,...,3.0,5,3,4,5,5,4,3,4,5653753.0
1,1,2008-02-14,Ankara,Big Cities,FC,4,5.0,4.0,4.0,1,...,3.0,0,0,0,0,0,0,0,0,6923131.0
2,2,2013-03-09,Diyarbakır,Other,IL,2,4.0,2.0,5.0,2,...,3.0,0,0,0,0,0,0,0,0,2055379.0
3,3,2012-02-02,Tokat,Other,IL,6,4.5,6.0,6.0,4,...,7.5,25,12,10,6,18,12,12,6,2675511.0
4,4,2009-05-09,Gaziantep,Other,IL,3,4.0,3.0,4.0,2,...,3.0,5,1,3,2,3,4,3,3,4316715.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,132,2008-06-25,Trabzon,Other,FC,2,3.0,3.0,5.0,4,...,3.0,0,0,0,0,0,0,0,0,5787594.0
133,133,2006-10-12,İzmir,Big Cities,FC,4,5.0,4.0,4.0,2,...,3.0,0,0,0,0,0,0,0,0,9262754.0
134,134,2006-07-08,Kayseri,Other,FC,3,4.0,4.0,4.0,2,...,3.0,0,0,0,0,0,0,0,0,2544857.0
135,135,2010-10-29,İstanbul,Big Cities,FC,4,5.0,4.0,5.0,2,...,3.0,0,0,0,0,0,0,0,0,7217634.0


In [4]:
df["Open Date"]

0     1999-07-17
1     2008-02-14
2     2013-03-09
3     2012-02-02
4     2009-05-09
         ...    
132   2008-06-25
133   2006-10-12
134   2006-07-08
135   2010-10-29
136   2009-09-01
Name: Open Date, Length: 137, dtype: datetime64[ns]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 43 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Id          137 non-null    int64         
 1   Open Date   137 non-null    datetime64[ns]
 2   City        137 non-null    object        
 3   City Group  137 non-null    object        
 4   Type        137 non-null    object        
 5   P1          137 non-null    int64         
 6   P2          137 non-null    float64       
 7   P3          137 non-null    float64       
 8   P4          137 non-null    float64       
 9   P5          137 non-null    int64         
 10  P6          137 non-null    int64         
 11  P7          137 non-null    int64         
 12  P8          137 non-null    int64         
 13  P9          137 non-null    int64         
 14  P10         137 non-null    int64         
 15  P11         137 non-null    int64         
 16  P12         137 non-null  

## Sort Datafrmae by Open Date

In [6]:
df.sort_values(by=["Open Date"], inplace=True, ascending=True)
df["Open Date"]

35    1996-05-08
122   1997-04-10
28    1998-01-28
112   1998-03-14
64    1998-07-13
         ...    
77    2013-10-25
52    2013-11-12
65    2013-12-21
119   2014-01-03
21    2014-01-25
Name: Open Date, Length: 137, dtype: datetime64[ns]

## Copy of original DataFrame

In [7]:
df_tmp = df.copy()

## Add datetime parameters for ```Open Date``` column

In [8]:
df_tmp["openYear"] = df_tmp["Open Date"].dt.year
df_tmp["openMonth"] = df_tmp["Open Date"].dt.month
df_tmp["openDay"] = df_tmp["Open Date"].dt.day

In [9]:
df_tmp.head()

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P32,P33,P34,P35,P36,P37,revenue,openYear,openMonth,openDay
35,35,1996-05-08,Adana,Other,IL,4,5.0,4.0,3.0,1,...,3,3,3,4,3,3,3903884.0,1996,5,8
122,122,1997-04-10,İstanbul,Big Cities,IL,12,7.5,6.0,7.5,2,...,25,6,18,3,16,6,4286645.0,1997,4,10
28,28,1998-01-28,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,1,...,5,2,3,5,4,4,4705945.0,1998,1,28
112,112,1998-03-14,İstanbul,Big Cities,IL,4,5.0,5.0,4.0,1,...,1,2,2,1,3,3,3784230.0,1998,3,14
64,64,1998-07-13,İstanbul,Big Cities,IL,12,7.5,6.0,6.0,2,...,25,4,24,15,20,6,2551252.0,1998,7,13


In [10]:
# Dropping ```Open Date``` Column
df_tmp.drop("Open Date", axis=1, inplace=True)

In [12]:
df_tmp.head().T

,35,122,28,112,64
Id,35,122,28,112,64
City,Adana,İstanbul,İstanbul,İstanbul,İstanbul
City Group,Other,Big Cities,Big Cities,Big Cities,Big Cities
Type,IL,IL,IL,IL,IL
P1,4,12,4,4,12
P2,5,7.5,5,5,7.5
P3,4,6,4,5,6
P4,3,7.5,4,4,6
P5,1,2,1,1,2
P6,2,8,3,5,8


In [11]:
df_tmp.City.value_counts()

İstanbul          50
Ankara            19
İzmir              9
Bursa              5
Samsun             5
Antalya            4
Sakarya            4
Tekirdağ           3
Diyarbakır         3
Adana              3
Eskişehir          3
Kayseri            3
Konya              2
Muğla              2
Aydın              2
Trabzon            2
Amasya             1
Şanlıurfa          1
Balıkesir          1
Tokat              1
Gaziantep          1
Uşak               1
Kırklareli         1
Kütahya            1
Isparta            1
Bolu               1
Elazığ             1
Edirne             1
Kastamonu          1
Afyonkarahisar     1
Karabük            1
Denizli            1
Osmaniye           1
Kocaeli            1
Name: City, dtype: int64

In [13]:
df_tmp.Type.value_counts()

FC    76
IL    60
DT     1
Name: Type, dtype: int64

In [14]:
df_tmp["City Group"].value_counts()

Big Cities    78
Other         59
Name: City Group, dtype: int64

In [15]:
df.isna().sum()

Id            0
Open Date     0
City          0
City Group    0
Type          0
P1            0
P2            0
P3            0
P4            0
P5            0
P6            0
P7            0
P8            0
P9            0
P10           0
P11           0
P12           0
P13           0
P14           0
P15           0
P16           0
P17           0
P18           0
P19           0
P20           0
P21           0
P22           0
P23           0
P24           0
P25           0
P26           0
P27           0
P28           0
P29           0
P30           0
P31           0
P32           0
P33           0
P34           0
P35           0
P36           0
P37           0
revenue       0
dtype: int64

## 5. Modelling

In [16]:
df_tmp["revenue"]

35     3903884.0
122    4286645.0
28     4705945.0
112    3784230.0
64     2551252.0
         ...    
77     1904842.0
52     1763231.0
65     1756069.0
119    3780019.0
21     1149870.0
Name: revenue, Length: 137, dtype: float64

In [17]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_jobs=-1,
                              random_state=42)

model.fit(df_tmp.drop("revenue", axis=1), df_tmp["revenue"])

ValueError: could not convert string to float: 'Adana'

In [18]:
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 137 entries, 35 to 21
Data columns (total 45 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Id          137 non-null    int64  
 1   City        137 non-null    object 
 2   City Group  137 non-null    object 
 3   Type        137 non-null    object 
 4   P1          137 non-null    int64  
 5   P2          137 non-null    float64
 6   P3          137 non-null    float64
 7   P4          137 non-null    float64
 8   P5          137 non-null    int64  
 9   P6          137 non-null    int64  
 10  P7          137 non-null    int64  
 11  P8          137 non-null    int64  
 12  P9          137 non-null    int64  
 13  P10         137 non-null    int64  
 14  P11         137 non-null    int64  
 15  P12         137 non-null    int64  
 16  P13         137 non-null    float64
 17  P14         137 non-null    int64  
 18  P15         137 non-null    int64  
 19  P16         137 non-null    i

## Convert strings to categories

In [19]:
pd.api.types.is_string_dtype(df_tmp["City"])

True

In [20]:
# find the column containing strings
for label, content in df_tmp.items():
    if  pd.api.types.is_string_dtype(content):
        print(label)

City
City Group
Type


In [21]:
# Turn strings to category values
for label, content in df_tmp.items():
    if pd.api.types.is_string_dtype(content):
        df_tmp[label] = content.astype("category").cat.as_ordered()

In [22]:
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 137 entries, 35 to 21
Data columns (total 45 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Id          137 non-null    int64   
 1   City        137 non-null    category
 2   City Group  137 non-null    category
 3   Type        137 non-null    category
 4   P1          137 non-null    int64   
 5   P2          137 non-null    float64 
 6   P3          137 non-null    float64 
 7   P4          137 non-null    float64 
 8   P5          137 non-null    int64   
 9   P6          137 non-null    int64   
 10  P7          137 non-null    int64   
 11  P8          137 non-null    int64   
 12  P9          137 non-null    int64   
 13  P10         137 non-null    int64   
 14  P11         137 non-null    int64   
 15  P12         137 non-null    int64   
 16  P13         137 non-null    float64 
 17  P14         137 non-null    int64   
 18  P15         137 non-null    int64   
 19  P16     

In [23]:
df_tmp.City.cat.categories

Index(['Adana', 'Afyonkarahisar', 'Amasya', 'Ankara', 'Antalya', 'Aydın',
       'Balıkesir', 'Bolu', 'Bursa', 'Denizli', 'Diyarbakır', 'Edirne',
       'Elazığ', 'Eskişehir', 'Gaziantep', 'Isparta', 'Karabük', 'Kastamonu',
       'Kayseri', 'Kocaeli', 'Konya', 'Kütahya', 'Kırklareli', 'Muğla',
       'Osmaniye', 'Sakarya', 'Samsun', 'Tekirdağ', 'Tokat', 'Trabzon', 'Uşak',
       'İstanbul', 'İzmir', 'Şanlıurfa'],
      dtype='object')

In [24]:
df_tmp.City.cat.codes

35      0
122    31
28     31
112    31
64     31
       ..
77      3
52     30
65     32
119    31
21     31
Length: 137, dtype: int8

All Data is now numeric, we should be able to build the machine learning model now.

In [25]:
%%time
model = RandomForestRegressor(n_jobs=-1,
                              random_state=42)
model.fit(df_tmp.drop("revenue", axis=1), df_tmp["revenue"])

ValueError: could not convert string to float: 'Adana'

In [26]:
df_tmp.isna().sum()

Id            0
City          0
City Group    0
Type          0
P1            0
P2            0
P3            0
P4            0
P5            0
P6            0
P7            0
P8            0
P9            0
P10           0
P11           0
P12           0
P13           0
P14           0
P15           0
P16           0
P17           0
P18           0
P19           0
P20           0
P21           0
P22           0
P23           0
P24           0
P25           0
P26           0
P27           0
P28           0
P29           0
P30           0
P31           0
P32           0
P33           0
P34           0
P35           0
P36           0
P37           0
revenue       0
openYear      0
openMonth     0
openDay       0
dtype: int64

## Overview

> With over 1,200 quick service restaurants across the globe, TFI is the company behind some of the world's most well-known brands: Burger King, Sbarro, Popeyes, Usta Donerci, and Arby’s. They employ over 20,000 people in Europe and Asia and make significant daily investments in developing new restaurant sites.
Right now, deciding when and where to open new restaurants is largely a subjective process based on the personal judgement and experience of development teams. This subjective data is difficult to accurately extrapolate across geographies and cultures.
New restaurant sites take large investments of time and capital to get up and running. When the wrong location for a restaurant brand is chosen, the site closes within 18 months and operating losses are incurred.
Finding a mathematical model to increase the effectiveness of investments in new restaurant sites would allow TFI to invest more in other important business areas, like sustainability, innovation, and training for new employees. Using demographic, real estate, and commercial data, this competition challenges you to predict the annual restaurant sales of 100,000 regional locations.


## Evaluation

### **Root Mean Squared Error (RMSE)**

Submissions are scored on the root mean squared error. RMSE is very common and is a suitable general-purpose error metric. Compared to the Mean Absolute Error, RMSE punishes large errors:
\[\textrm{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2},\]
where y hat is the predicted value and y is the original value.